In [1]:
# ABANDON
"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record
"""

'\nUsage:\n  # From tensorflow/models/\n  # Create train data:\n  python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record\n\n  # Create test data:\n  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record\n'

In [4]:
import sys

tensorflow_path = "/home/qiaohe/models"
sys.path.append(tensorflow_path + "/research")
sys.path.append(tensorflow_path + "/research/object_detection")
sys.path.append(tensorflow_path + "/research/object_detection/utils")

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


In [5]:
base_dir = "/home/qiaohe/hand_pose"
csv_train_input= os.path.join(base_dir, 'train_labels.csv')
csv_test_input= os.path.join(base_dir, 'test_labels.csv')
csv_train_output=os.path.join(base_dir, 'train.record')
csv_test_output=os.path.join(base_dir, 'test.record')
img_path = os.path.join(base_dir, 'Color')

In [6]:
# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == "hand":
        return 1
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [9]:
writer = tf.python_io.TFRecordWriter(csv_train_output)
print("test:", csv_train_output,csv_train_input, writer)
examples = pd.read_csv(csv_train_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, img_path)
    writer.write(tf_example.SerializeToString())

writer.close()
print('Successfully created the TFRecords: {}'.format(csv_train_output))

test: /home/qiaohe/hand_pose/train.record /home/qiaohe/hand_pose/train_labels.csv <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x7efdcdae6630>
Successfully created the TFRecords: /home/qiaohe/hand_pose/train.record


In [10]:
writer = tf.python_io.TFRecordWriter(csv_test_output)
print("test:", csv_test_output,csv_test_input, writer)
examples = pd.read_csv(csv_test_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, img_path)
    writer.write(tf_example.SerializeToString())

writer.close()
print('Successfully created the TFRecords: {}'.format(csv_test_output))

test: /home/qiaohe/hand_pose/test.record /home/qiaohe/hand_pose/test_labels.csv <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x7efdc5a8b358>
Successfully created the TFRecords: /home/qiaohe/hand_pose/test.record
